# Power Plant Capture Prices

<br>

### Imports

In [1]:
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd

import os
import warnings
from tqdm import tqdm

<br>

### User Inputs

In [2]:
static_data_dir = 'C:/Users/Ayrto/Desktop/Freelance Work/FEA/work/data-hub/data/static'
powerdict_fp = 'C:/Users/Ayrto/Desktop/Side Projects/Power-Station-Dictionary/data/dictionary/ids.csv'

In [3]:
@dask.delayed
def read_B1610_file(filename, columns):
    df_B1610_week = pd.read_csv(filename)
    cols_to_add = list(set(columns) - set(df_B1610_week.columns))
    df_B1610_week[cols_to_add] = np.NaN
    df_B1610_week = df_B1610_week[columns]
    
    return df_B1610_week

def get_B1610_columns(static_dir, source_name='bmrs', stream='B1610'):
    B1610_files = [f for f in os.listdir(f'{static_dir}/{source_name}/{stream}') if '.csv' in f]

    columns = []

    for B1610_file in tqdm(B1610_files):
        df_B1610_week = pd.read_csv(f'{static_dir}/{source_name}/{stream}/{B1610_file}')
        columns += list(df_B1610_week.columns)

    columns = ['datetime'] + sorted(list(set(columns)-set(['datetime'])))

    return columns

def load_B1610_dask_stream_df(static_dir, source_name, stream, dt_col='datetime'):
    # Identifying columns
    columns = get_B1610_columns(static_dir)
    
    # Loading data
    B1610_files = [f for f in os.listdir(f'{static_dir}/{source_name}/{stream}') if '.csv' in f]
    df_B1610 = dd.from_delayed([read_B1610_file(f'{static_dir}/{source_name}/{stream}/{B1610_file}', columns) for B1610_file in B1610_files])

    # Formatting date index
    if dt_col is not None:
        df_B1610[dt_col] = df_B1610[dt_col].map(lambda dt: pd.to_datetime(dt, format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True))
        df_B1610 = df_B1610.set_index(dt_col)
    
    return df_B1610

In [4]:
%%time

source_name = 'bmrs'
stream = 'B1610'

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    df_B1610 = load_B1610_dask_stream_df(static_data_dir, source_name, stream).compute()   

df_B1610.head(3)

100%|████████████████████████████████████████████████████████████████████████████████| 288/288 [00:07<00:00, 37.18it/s]


Wall time: 2min 10s


,ABRBO-1,ABRTW-1,ABTH7,ABTH7G,ABTH8,ABTH8G,ABTH9,ABTH9G,ACHRW-1,AKGLW-2,...,WILCT-1,WLNYO-2,WLNYO-3,WLNYO-4,WLNYW-1,WTMSO-1,WYLF-1,WYLF-2,WYLF-3,WYLF-4
datetime,,,,,,,,,,,,,,,,,,,,,
2015-10-04 23:00:00+00:00,NaN,NaN,421.692,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,132.90,NaN,NaN,95.602,9.858,227.380,212.41,NaN,NaN
2015-10-04 23:30:00+00:00,NaN,NaN,425.096,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,143.02,NaN,NaN,107.324,14.890,227.470,212.53,NaN,NaN
2015-10-05 00:00:00+00:00,NaN,NaN,423.292,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,158.98,NaN,NaN,118.342,23.074,227.666,212.72,NaN,NaN


In [5]:
df_powerdict = pd.read_csv(powerdict_fp)

df_powerdict.head(3)

,dictionary_id,gppd_idnr,esail_id,name,sett_bmu_id,ngc_bmu_id,4c_offshore_id,windpowernet_id,wikidata_id,wikipedia_id,power_technology_id,eutl_id
0,10000,NaN,MARK,Rothes Bio-Plant CHP,"E_MARK-1, E_MARK-2","MARK-1, MARK-2",NaN,NaN,NaN,NaN,NaN,NaN
1,10001,"GBR1000377, GBR1000369",DIDC,Didcot,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3, T_DIDC1G, ...","DIDC1, DIDC2, DIDC4, DIDC3, DIDC1G, DIDC2G, DI...",NaN,NaN,NaN,NaN,NaN,97165
2,10002,"GBR1000374, GBR1000375",ABTH,Aberthaw B,"T_ABTH7, T_ABTH8, T_ABTH9, T_ABTH7G, T_ABTH8G,...","ABTH7, ABTH8, ABTH9, ABTH7G, ABTH8G, ABTH9G",NaN,NaN,NaN,NaN,NaN,97175


In [6]:
#exports
def retrieve_ei_df(
    start_year: int=2009, 
    end_year: int=2021
):
    construct_ei_year_url = lambda year: f'https://raw.githubusercontent.com/AyrtonB/Electric-Insights/master/data/electric_insights_{year}.csv'
    
    ei_dfs = []

    for year in tqdm(range(start_year, end_year+1)):
        ei_year_url = construct_ei_year_url(year)
        df_ei_year = pd.read_csv(ei_year_url)
        ei_dfs += [df_ei_year]

    df_ei = pd.concat(ei_dfs)

    df_ei['local_datetime'] = pd.to_datetime(df_ei['local_datetime'], utc=True)
    df_ei = df_ei.set_index('local_datetime')
    df_ei = df_ei.loc[~df_ei.index.duplicated()]
    
    return df_ei

In [7]:
df_ei = retrieve_ei_df()

df_ei.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:08<00:00,  1.45it/s]


,day_ahead_price,SP,imbalance_price,valueSum,temperature,TCO2_per_h,gCO2_per_kWh,nuclear,biomass,coal,...,demand,pumped_storage,wind_onshore,wind_offshore,belgian,dutch,french,ireland,northern_ireland,irish
local_datetime,,,,,,,,,,,,,,,,,,,,,
2009-01-01 00:00:00+00:00,58.05,1.0,74.74,74.74,-0.6,21278.0,555.0,6.973,0.0,17.650,...,38.329,-0.404,NaN,NaN,0.0,0.0,1.977,0.0,0.0,-0.161
2009-01-01 00:30:00+00:00,56.33,2.0,74.89,74.89,-0.6,21442.0,558.0,6.968,0.0,17.770,...,38.461,-0.527,NaN,NaN,0.0,0.0,1.977,0.0,0.0,-0.160
2009-01-01 01:00:00+00:00,52.98,3.0,76.41,76.41,-0.6,21614.0,569.0,6.970,0.0,18.070,...,37.986,-1.018,NaN,NaN,0.0,0.0,1.977,0.0,0.0,-0.160
2009-01-01 01:30:00+00:00,50.39,4.0,37.73,37.73,-0.6,21320.0,578.0,6.969,0.0,18.022,...,36.864,-1.269,NaN,NaN,0.0,0.0,1.746,0.0,0.0,-0.160
2009-01-01 02:00:00+00:00,48.70,5.0,59.00,59.00,-0.6,21160.0,585.0,6.960,0.0,17.998,...,36.180,-1.566,NaN,NaN,0.0,0.0,1.730,0.0,0.0,-0.160


In [8]:
flatten_list = lambda list_: [item for sublist in list_ for item in sublist]

powerdict_ngc_bmu_ids = sorted(list(set(flatten_list(df_powerdict['ngc_bmu_id'].dropna().str.split(', ').to_list()))))

len(powerdict_ngc_bmu_ids)

444

In [9]:
B1610_ngc_bmu_ids = sorted(list(df_B1610.columns))

missing_ngc_bmu_ids_from_powerdict = sorted(list(set(B1610_ngc_bmu_ids) - set(powerdict_ngc_bmu_ids)))

missing_ngc_bmu_ids_from_powerdict

['GRAI1G', 'GRAI4G', 'KNLCV-1', 'LCSMH-1', 'SEEL-1', 'WILCT-1']

In [35]:
common_ngc_bmu_ids = sorted(list(set(B1610_ngc_bmu_ids).intersection(set(powerdict_ngc_bmu_ids))))

start_year = 2016
end_year = 2020

s_dap = df_ei['day_ahead_price']
capture_price_data = []

for id_ in tqdm(common_ngc_bmu_ids):
    for year in range(start_year, end_year+1):
        start_dt = pd.to_datetime(year, format='%Y', utc=True)
        end_dt = pd.to_datetime(year+1, format='%Y', utc=True)-pd.Timedelta(minutes=30)
        
        s_weights = df_B1610.loc[start_dt:end_dt, id_].fillna(0)
        s_dap_year = s_dap[start_dt:end_dt].dropna()
        
        common_dt_idxs = s_weights.index.intersection(s_dap_year.index)

        if s_weights.sum() != 0:
            capture_price = np.average(s_dap.loc[common_dt_idxs], weights=s_weights.loc[common_dt_idxs])
        else:
            capture_price = np.nan

        capture_price_data += [[id_, year, capture_price]]

s_capture_prices = (
    pd.DataFrame(capture_price_data, columns=['ngc_bmu_id', 'year', 'capture_price'])
    .set_index(['ngc_bmu_id', 'year'])
    ['capture_price']
    .dropna()
)

s_capture_prices.head()

ngc_bmu_id  year
ABRBO-1     2019    39.429610
            2020    32.759415
ABRTW-1     2019    39.711074
            2020    32.610071
ABTH7       2016    42.271283
Name: capture_price, dtype: float64

In [38]:
save_data = False

if save_data == True:
    s_capture_prices.reset_index().to_csv('../datasets/capture-prices/capture-prices.csv', index=False)